In [ ]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'ClusterCast.settings'
import test_utils
import django
django.setup()
import sys
sys.path.append("/home/ajp031/StockDeepLearning/ClusterCast/ClusterCast")
from asgiref.sync import sync_to_async
from importlib import reload
import ClusterPipeline.models.ClusterProcessing as cp
import ClusterPipeline.models.SequencePreprocessing as sp
from copy import deepcopy
import numpy as np
from tensorflow.keras.backend import clear_session
import random

In [ ]:
reload(test_utils)

In [ ]:
group_ids = [38]
groups = await test_utils.extract_cluster_groups(group_ids)

In [ ]:
clusters = await test_utils.get_all_clusters(groups)

cur_group = groups[0]

In [ ]:
cluster_of_interest_id = 435
cluster_of_interest = [cluster for cluster in clusters if cluster.id == cluster_of_interest_id][0]

In [ ]:
features = groups[0].group_params.training_features

features

In [ ]:
X_train, y_train, X_test, y_test = cluster_of_interest.get_3d_array()


In [ ]:
target_features = [] 
target_features += ['pctChgclose{}_target'.format(i) for i in range(-14, 0) ]
target_features += ['pctChgclose-0_target']
target_features += ['pctChgclose+{}_target'.format(i) for i in range(1, 7) ]
training_features = random.sample(features, 30)
print(training_features)

In [ ]:
reload(test_utils)

In [ ]:
print(target_features)
print(len(target_features))

In [ ]:
y_feature_dict = cur_group.y_feature_dict
X_feature_dict = cur_group.X_feature_dict

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_feature_dict)

In [ ]:

X_train_filtered = test_utils.filter_by_features(X_train, training_features, X_feature_dict)
X_test_filtered = test_utils.filter_by_features(X_test, training_features, X_feature_dict)
y_train_filtered = test_utils.filter_y_by_features(y_train, target_features, y_feature_dict)
y_test_filtered = test_utils.filter_y_by_features(y_test, target_features, y_feature_dict)
print(X_train_filtered.shape)
print(y_train_filtered.shape)
print(X_test_filtered.shape)
print(y_test_filtered.shape)

In [ ]:
attention_model = test_utils.create_attention_model(input_steps=X_train.shape[1],output_steps=len(target_features),features=len(training_features))


In [ ]:
trainined_attention_model = test_utils.train_model(attention_model, (X_train, y_train, X_test, y_test), training_features, 
                                                   target_features, X_feature_dict, y_feature_dict,
                                                   epochs = 100, batch_size = 32, lr = 0.001, early_stopping_patience = 15,
                                                   loss = 'mse')

In [ ]:
attention_model_accuracy, attention_results, predicted_y = test_utils.eval_model(X_test_filtered, y_test_filtered, trainined_attention_model)

In [ ]:
reload(test_utils)

In [ ]:
print(attention_model_accuracy)
test_utils.visualize_future_distribution(attention_results)

In [ ]:
sequence_element = cluster_of_interest.test_seq_elements[0]
print(sequence_element.y_feature_sets)
sp.SequenceElement.visualize_future_movement(sequence_element, attention_results, isCuma = False)